In [2]:
%load_ext autoreload
%autoreload 2

%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision
import torchvision.models as models
from torchvision import transforms
from datasets import load_dataset, concatenate_datasets

/Users/dylanmace/miniforge3/envs/earlyexit/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Check for CUDA support
print(f"Is CUDA available? {torch.cuda.is_available()}")

# Set the device
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using device: {device}")


PyTorch version: 2.1.0
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Is CUDA available? False
Using device: mps


In [5]:
resnet50 = models.resnet50(pretrained=True)
in_features = resnet50.fc.in_features
resnet50.fc = nn.Linear(in_features, 10)

# # freeze all layers
# for param in resnet50.parameters():
#     param.requires_grad = False

/Users/dylanmace/miniforge3/envs/earlyexit/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/dylanmace/miniforge3/envs/earlyexit/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
from EarlyExitModel import EarlyExitModel

model = EarlyExitModel(resnet50, 10, device)
model

EarlyExitModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
     

In [7]:
model.clear_exits()
exit_layers = [model.add_exit(layer) for layer in ('layer1', 'layer2', 'layer3')]
model.to(device)  # Move the model to the selected device
model

EarlyExitModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): OptionalExitModule(
      (module): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(i

In [8]:
from DataLoader import CustomDataset

hf_dataset = load_dataset("frgfm/imagenette", '320px')
hf_dataset = concatenate_datasets(hf_dataset.values())

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


torch_dataset = CustomDataset(hf_dataset, transform=transform)

batch_size = 32

test_size = 0.2
test_volume = int(test_size * len(torch_dataset))
train_volume = len(torch_dataset) - test_volume

train_dataset, test_dataset = random_split(torch_dataset, [train_volume, test_volume])
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=False, 
    num_workers=4
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4
)

## Early Exit Model Training

In [9]:
from EarlyExitTrainer import ModelTrainer

trainer = ModelTrainer(model, device)

trainer.train(train_dataloader, epoch_count=1, validation_loader=test_dataloader)


Training early exit layer 1
Beginning epoch 0


Epoch 0 Loss 0.12589962091018905
Epoch 0 Accuracy 0.0016791044776119403


Epoch 0 Validation Loss 3.9433146033968245
Epoch 0 Validation Accuracy 0.10443722943722944
Training early exit layer 2
Beginning epoch 0


Epoch 0 Loss 0.16908268145660857
Epoch 0 Accuracy 0.0018656716417910447


Epoch 0 Validation Loss 11.77011867931911
Epoch 0 Validation Accuracy 0.09949945887445887
Training early exit layer 3
Beginning epoch 0


...

Epoch 0:  78%|████████████████████████████▏       | 262/335 [01:35<00:22,  3.25it/s, Accuracy=0.688]

[None, None, None, tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
       device='mps:0')]
Optimizing Gate Layer 0
Optimizing Gate Layer 1
Optimizing Gate Layer 2


Epoch 0:  79%|████████████████████████████▎       | 263/335 [01:36<00:22,  3.25it/s, Accuracy=0.875]

[None, None, None, tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
       device='mps:0')]
Optimizing Gate Layer 0
Optimizing Gate Layer 1
Optimizing Gate Layer 2


Epoch 0:  79%|████████████████████████████▎       | 264/335 [01:36<00:21,  3.26it/s, Accuracy=0.688]

In [ ]:
# import time

# # TODO: replace this with a transfer learned resnet model

# original_model = models.resnet50(pretrained=True)

# nonEarlyExitModel = EarlyExitModel(original_model, 1000, device)
# nonEarlyExitModel.to(device)
# nonModelTrainer = ModelTrainer(nonEarlyExitModel, device)

# # validate the model
# print("Validating original Resnet model")
# start = time.time()
# loss, acc, exits = nonModelTrainer.validate(test_dataloader)
# end = time.time()
# print(f"Validation Loss: {loss}, Validation Accuracy: {acc}")
# print(f"Validation time: {end - start}")
# print("=====================================================")

# # validate the new early exit model
# print("Validating new ResnetEE model")
# start = time.time()
# loss, acc = trainer.validate(test_dataloader)
# end = time.time()
# print(f"Validation Loss: {loss}, Validation Accuracy: {acc}")
# print(f"Average exit index: {exits}")
# print(f"Validation time: {end - start}")

Validating original Resnet model


KeyboardInterrupt: 